In [1]:
import sys
import re
import numpy as np
import ast

In [2]:
sys.path.append('MLSTM-FCN/')

In [3]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, multiply, concatenate, Activation, Masking, Reshape
from keras.layers import Conv1D, BatchNormalization, GlobalAveragePooling1D, Permute, Dropout

from utils.constants import MAX_NB_VARIABLES, NB_CLASSES_LIST, MAX_TIMESTEPS_LIST
from utils.keras_utils import train_model, evaluate_model, set_trainable
from utils.layer_utils import AttentionLSTM

Using TensorFlow backend.
/home/paperspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/paperspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/paperspace/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/paperspace/.local/lib/python3.6/site-packages/t

In [4]:
resultsFilename='results_data_FINAL.txt'

In [5]:
DATASET_INDEX = 60

MAX_TIMESTEPS = MAX_TIMESTEPS_LIST[DATASET_INDEX]
MAX_NB_VARIABLES = MAX_NB_VARIABLES[DATASET_INDEX]
NB_CLASS = NB_CLASSES_LIST[DATASET_INDEX]

TRAINABLE = True

In [7]:
def generate_model_2():
    ip = Input(shape=(MAX_NB_VARIABLES, MAX_TIMESTEPS))
    # stride = 10

    # x = Permute((2, 1))(ip)
    # x = Conv1D(MAX_NB_VARIABLES // stride, 8, strides=stride, padding='same', activation='relu', use_bias=False,
    #            kernel_initializer='he_uniform')(x)  # (None, variables / stride, timesteps)
    # x = Permute((2, 1))(x)

    #ip1 = K.reshape(ip,shape=(MAX_TIMESTEPS,MAX_NB_VARIABLES))
    #x = Permute((2, 1))(ip)
    x = Masking()(ip)
    x = AttentionLSTM(8)(x)
    x = Dropout(0.8)(x)

    y = Permute((2, 1))(ip)
    y = Conv1D(128, 8, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(256, 5, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = squeeze_excite_block(y)

    y = Conv1D(128, 3, padding='same', kernel_initializer='he_uniform')(y)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)

    y = GlobalAveragePooling1D()(y)

    x = concatenate([x, y])

    out = Dense(NB_CLASS, activation='sigmoid')(x)

    model = Model(ip, out)
    model.summary()

    # add load model code here to fine-tune

    return model

In [10]:
def squeeze_excite_block(input):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    filters = input._keras_shape[-1] # channel_axis = -1 for TF

    se = GlobalAveragePooling1D()(input)
    se = Reshape((1, filters))(se)
    se = Dense(filters // 16,  activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    se = multiply([input, se])
    return se

In [11]:
n_epochs=250

In [12]:
with open(resultsFilename, "a") as text_file:
    print(f"Current Fold: Only", file=text_file)
model = generate_model_2()
train_model(model, DATASET_INDEX, dataset_prefix='eeg2_attention', epochs=n_epochs, batch_size=128)
evaluate_model(model, DATASET_INDEX, dataset_prefix='eeg2_attention', batch_size=128)



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 255)      0                                            
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 255, 64)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 255, 128)     65664       permute_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 255, 128)     512         conv1d_1[0][0]                   
__________________________________________________________________________________________________
activation

 - 21s - loss: 0.3575 - acc: 0.8417 - f1_m: 0.8262 - precision_m: 0.8325 - recall_m: 0.8200 - tp_m: 100.1600 - fp_m: 20.1867 - tn_m: 101.9467 - fn_m: 21.9733 - val_loss: 0.6699 - val_acc: 0.7083 - val_f1_m: 0.6520 - val_precision_m: 0.7022 - val_recall_m: 0.6100 - val_tp_m: 74.9467 - val_fp_m: 29.8400 - val_tn_m: 92.2933 - val_fn_m: 47.1867

Epoch 00004: loss improved from 0.42398 to 0.35751, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 21.199344396591187
Epoch 5/250
epoch time start: 1578300003.3078496
 - 21s - loss: 0.3030 - acc: 0.8717 - f1_m: 0.8597 - precision_m: 0.8614 - recall_m: 0.8583 - tp_m: 104.9333 - fp_m: 17.0400 - tn_m: 105.0933 - fn_m: 17.2000 - val_loss: 0.5223 - val_acc: 0.7533 - val_f1_m: 0.6853 - val_precision_m: 0.7578 - val_recall_m: 0.6283 - val_tp_m: 77.6933 - val_fp_m: 23.7600 - val_tn_m: 98.3733 - val_fn_m: 44.4400

Epoch 00005: loss improved from 0.35751 to 0.30296, saving model to ./weights/eeg2_attention_weights.h5
epoch time meas


Epoch 00019: loss improved from 0.03090 to 0.02101, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 21.21800136566162
Epoch 20/250
epoch time start: 1578300322.6443748
 - 22s - loss: 0.0171 - acc: 1.0000 - f1_m: 0.9822 - precision_m: 1.0000 - recall_m: 0.9650 - tp_m: 117.8533 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 4.2800 - val_loss: 0.3633 - val_acc: 0.8517 - val_f1_m: 0.7963 - val_precision_m: 0.9254 - val_recall_m: 0.7000 - val_tp_m: 85.3333 - val_fp_m: 7.0667 - val_tn_m: 115.0667 - val_fn_m: 36.8000

Epoch 00020: loss improved from 0.02101 to 0.01707, saving model to ./weights/eeg2_attention_weights.h5
epoch time measured: 21.61645197868347
Epoch 21/250
epoch time start: 1578300344.2610278
 - 21s - loss: 0.0129 - acc: 1.0000 - f1_m: 0.9890 - precision_m: 1.0000 - recall_m: 0.9783 - tp_m: 119.5600 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 2.5733 - val_loss: 0.3493 - val_acc: 0.8583 - val_f1_m: 0.7969 - val_precision_m: 0.9211 - val_recall_m: 0.7033 - 


Epoch 00035: loss did not improve
epoch time measured: 21.858596324920654
Epoch 36/250
epoch time start: 1578300664.0108562
 - 21s - loss: 0.0028 - acc: 1.0000 - f1_m: 0.9975 - precision_m: 1.0000 - recall_m: 0.9950 - tp_m: 121.5600 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.5733 - val_loss: 0.3243 - val_acc: 0.8867 - val_f1_m: 0.8181 - val_precision_m: 0.9439 - val_recall_m: 0.7233 - val_tp_m: 88.0533 - val_fp_m: 5.4133 - val_tn_m: 116.7200 - val_fn_m: 34.0800

Epoch 00036: loss did not improve
epoch time measured: 21.24206852912903
Epoch 37/250
epoch time start: 1578300685.2530398
 - 21s - loss: 0.0020 - acc: 1.0000 - f1_m: 0.9975 - precision_m: 1.0000 - recall_m: 0.9950 - tp_m: 121.4933 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.6400 - val_loss: 0.3699 - val_acc: 0.8617 - val_f1_m: 0.8093 - val_precision_m: 0.9404 - val_recall_m: 0.7133 - val_tp_m: 86.6400 - val_fp_m: 5.4133 - val_tn_m: 116.7200 - val_fn_m: 35.4933

Epoch 00037: loss improved from 0.00272 to 0.00205, saving m

 - 21s - loss: 0.0012 - acc: 1.0000 - f1_m: 0.9983 - precision_m: 1.0000 - recall_m: 0.9967 - tp_m: 121.7067 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.4267 - val_loss: 0.3777 - val_acc: 0.8750 - val_f1_m: 0.8214 - val_precision_m: 0.9477 - val_recall_m: 0.7267 - val_tp_m: 88.2133 - val_fp_m: 4.9867 - val_tn_m: 117.1467 - val_fn_m: 33.9200

Epoch 00052: loss did not improve
epoch time measured: 21.068852424621582
Epoch 53/250
epoch time start: 1578301025.9622111
 - 21s - loss: 0.0013 - acc: 1.0000 - f1_m: 0.9983 - precision_m: 1.0000 - recall_m: 0.9967 - tp_m: 121.7067 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.4267 - val_loss: 0.3936 - val_acc: 0.8733 - val_f1_m: 0.8217 - val_precision_m: 0.9454 - val_recall_m: 0.7283 - val_tp_m: 88.4267 - val_fp_m: 5.2000 - val_tn_m: 116.9333 - val_fn_m: 33.7067

Epoch 00053: loss did not improve
epoch time measured: 21.26231861114502
Epoch 54/250
epoch time start: 1578301047.2246482
 - 21s - loss: 0.0012 - acc: 1.0000 - f1_m: 0.9992 - precisio

...


Epoch 00186: loss did not improve
epoch time measured: 21.157817363739014
Epoch 187/250
epoch time start: 1578303871.8670244
 - 21s - loss: 1.4930e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 122.1333 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.0000e+00 - val_loss: 0.4874 - val_acc: 0.8800 - val_f1_m: 0.8338 - val_precision_m: 0.9381 - val_recall_m: 0.7517 - val_tp_m: 91.2800 - val_fp_m: 6.0533 - val_tn_m: 116.0800 - val_fn_m: 30.8533

Epoch 00187: loss did not improve
epoch time measured: 21.03899097442627
Epoch 188/250
epoch time start: 1578303892.9062316
 - 21s - loss: 1.0194e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 122.1333 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.0000e+00 - val_loss: 0.4827 - val_acc: 0.8767 - val_f1_m: 0.8346 - val_precision_m: 0.9403 - val_recall_m: 0.7517 - val_tp_m: 91.2800 - val_fp_m: 5.8400 - val_tn_m: 116.2933 - val_fn_m: 30.8533

Epoch 00188: loss improved from 0.00011 to


Epoch 00203: loss did not improve
epoch time measured: 21.577107667922974
Epoch 204/250
epoch time start: 1578304231.9280012
 - 21s - loss: 1.2374e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 122.1333 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.0000e+00 - val_loss: 0.4846 - val_acc: 0.8850 - val_f1_m: 0.8390 - val_precision_m: 0.9434 - val_recall_m: 0.7567 - val_tp_m: 91.9200 - val_fp_m: 5.5600 - val_tn_m: 116.5733 - val_fn_m: 30.2133

Epoch 00204: loss did not improve
epoch time measured: 21.32392120361328
Epoch 205/250
epoch time start: 1578304253.2520163
 - 21s - loss: 2.0375e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 122.1333 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.0000e+00 - val_loss: 0.4834 - val_acc: 0.8817 - val_f1_m: 0.8376 - val_precision_m: 0.9427 - val_recall_m: 0.7550 - val_tp_m: 91.7067 - val_fp_m: 5.6267 - val_tn_m: 116.5067 - val_fn_m: 30.4267

Epoch 00205: loss did not improve
epoch ti


Epoch 00220: loss did not improve
epoch time measured: 21.17819118499756
Epoch 221/250
epoch time start: 1578304591.853635
 - 21s - loss: 8.6347e-05 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 122.1333 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.0000e+00 - val_loss: 0.4880 - val_acc: 0.8817 - val_f1_m: 0.8386 - val_precision_m: 0.9451 - val_recall_m: 0.7550 - val_tp_m: 91.7067 - val_fp_m: 5.3467 - val_tn_m: 116.7867 - val_fn_m: 30.4267

Epoch 00221: loss did not improve
epoch time measured: 20.99575662612915
Epoch 222/250
epoch time start: 1578304612.8495347
 - 21s - loss: 7.2727e-05 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 122.1333 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.0000e+00 - val_loss: 0.4902 - val_acc: 0.8850 - val_f1_m: 0.8395 - val_precision_m: 0.9452 - val_recall_m: 0.7567 - val_tp_m: 91.9200 - val_fp_m: 5.3467 - val_tn_m: 116.7867 - val_fn_m: 30.2133

Epoch 00222: loss improved from 0.00008 to 0


Epoch 00237: loss did not improve
epoch time measured: 20.981904983520508
Epoch 238/250
epoch time start: 1578304950.782273
 - 21s - loss: 7.7932e-05 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 122.1333 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.0000e+00 - val_loss: 0.4990 - val_acc: 0.8833 - val_f1_m: 0.8364 - val_precision_m: 0.9429 - val_recall_m: 0.7533 - val_tp_m: 91.4933 - val_fp_m: 5.5600 - val_tn_m: 116.5733 - val_fn_m: 30.6400

Epoch 00238: loss did not improve
epoch time measured: 21.06362271308899
Epoch 239/250
epoch time start: 1578304971.846029
 - 21s - loss: 8.4860e-05 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - tp_m: 122.1333 - fp_m: 0.0000e+00 - tn_m: 122.1333 - fn_m: 0.0000e+00 - val_loss: 0.5005 - val_acc: 0.8817 - val_f1_m: 0.8364 - val_precision_m: 0.9429 - val_recall_m: 0.7533 - val_tp_m: 91.4933 - val_fp_m: 5.5600 - val_tn_m: 116.5733 - val_fn_m: 30.6400

Epoch 00239: loss did not improve
epoch time

(0.8566666666666667, 0.5016763567924499)